
## **LSA WITH COSINE SIMILARITY**


**The below code is plagiarism detection implementation without explicit preprocessing function on custom dataset**

In [8]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import string
import warnings
warnings.filterwarnings('ignore')


def calculate_similarity(text1, text2):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    svd = TruncatedSVD(n_components=2)
    if tfidf_matrix.shape == (2, 1):
      return None
    lsa_matrix = svd.fit_transform(tfidf_matrix)
    similarity = cosine_similarity(lsa_matrix[0].reshape(1, -1), lsa_matrix[1].reshape(1, -1))[0][0]
    return similarity

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/plagiarism_dataset.csv')

# similarity threshold
threshold = 0.5

# Calculate similarity and predict plagiarism
predictions = []

#print(df.head(2))
ignored_idx = []

for index, row in df.iterrows():
    similarity = calculate_similarity(row['text1'], row['text2'])
    if not similarity:
      df.at[index,'is_plagiarized'] = np.nan
      continue
    is_plagiarized = int(similarity > threshold)
    predictions.append(is_plagiarized)


# Evaluate model
predicted = np.array(predictions)
actual = df['is_plagiarized'].dropna()
actual=actual.values

accuracy = accuracy_score(actual, predicted)
f1 = f1_score(actual, predicted)

Text1="The quick brown fox jumps over the lazy dog."
Text2="A swift fox of brown color leaps above a dog that is not active."
similarity = calculate_similarity(Text1, Text2)
is_plagiarized = int(similarity > threshold)
similarity_percentage = similarity * 100
output = 'Plagiarized' if is_plagiarized == 1 else 'Not Plagiarized'
print(f"Similarity for the two input documents is {output} with a similarity score of: {similarity_percentage:.2f}%")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


Similarity for the two input documents is Not Plagiarized with a similarity score of: 30.41%
Accuracy: 1.0
F1 Score: 1.0


The below code is plagiarism detection implementation with explicit preprocessing function on custom dataset along with additional lemmatization handled by TfidfVectorizer

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import string
import warnings
warnings.filterwarnings('ignore')

def preprocess(sentence):
    """Preprocess the sentence: lowercase, remove punctuation, tokenize."""
    sentence = sentence.lower()
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    return ''.join(sentence)

def calculate_similarity_and_common_words(text1, text2):
    """Calculate cosine similarity between two texts and identify common high-scoring words."""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

    # Find common words with the highest TF-IDF scores in both texts
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores_1 = tfidf_matrix[0].toarray().flatten()
    tfidf_scores_2 = tfidf_matrix[1].toarray().flatten()

    # Filter words by setting a threshold or consider all with non-zero in both
    common_words_scores = {feature_names[i]: min(tfidf_scores_1[i], tfidf_scores_2[i])
                           for i in range(len(feature_names))
                           if tfidf_scores_1[i] > 0 and tfidf_scores_2[i] > 0}

    # Sort by score
    sorted_common_words = sorted(common_words_scores.items(), key=lambda x: x[1], reverse=True)

    return similarity, sorted_common_words




# Load dataset
df = pd.read_csv('/content/drive/MyDrive/plagiarism_dataset.csv')

# Define similarity threshold
threshold = 0.5

# Calculate similarity and predict plagiarism
predictions = []

#print(df.head(2))
ignored_idx = []

for index, row in df.iterrows():
    similarity,_ = calculate_similarity_and_common_words(row['text1'], row['text2'])
    if not similarity:
      df.at[index,'is_plagiarized'] = np.nan
      continue
    is_plagiarized = int(similarity > threshold)
    predictions.append(is_plagiarized)


# Evaluate model
predicted = np.array(predictions)
actual = df['is_plagiarized'].dropna()
actual=actual.values

accuracy = accuracy_score(actual, predicted)
f1 = f1_score(actual, predicted)

Text1 = "This is a simple test document."
Text2 = "This document is a test, not a real document."
preprocessed_text1 = preprocess(Text1)
preprocessed_text2 = preprocess(Text2)

print(preprocessed_text1)
print(preprocessed_text2)
similarity, common_words = calculate_similarity_and_common_words(preprocessed_text1, preprocessed_text2)
similarity_percentage = similarity * 100
# Printing results
print(f"Similarity: {similarity_percentage:.2f}%")

if common_words:
    print("Common words contributing to similarity:")
    for word, score in common_words:
        print(f"{word}")
else:
    print("No common high-scoring words identified.")

this is a simple test document
this document is a test not a real document
Similarity: 61.81%
Common words contributing to similarity:
document
is
test
this


The below code is plagiarism detection implementation with explicit preprocessing function on custom dataset along with additional lemmatization handled by TfidfVectorizer and POS Tagging

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import string
import warnings
import re
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings('ignore')

default_similarity_value = 1;

# Download NLTK data (you may comment these out after the first run)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Initialize NLTK stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


# Helper function to map NLTK's POS tags to the format used by WordNetLemmatizer
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return 'n'

# Function for basic text preprocessing
def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters
    tokens = word_tokenize(text)  # Tokenize text

    tagged_tokens = pos_tag(tokens)  # Get POS tags for tokens

    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged_tokens if word not in stop_words]
    return ' '.join(tokens)

def calculate_similarity(text1, text2):
    """Calculate cosine similarity between two texts."""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    svd = TruncatedSVD(n_components=2)
    if tfidf_matrix.shape == (2, 1):
      return None
    lsa_matrix = svd.fit_transform(tfidf_matrix)
    similarity = cosine_similarity(lsa_matrix[0].reshape(1, -1), lsa_matrix[1].reshape(1, -1))[0][0]
    return similarity

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/plagiarism_dataset.csv')

# Define similarity threshold
threshold = 0.5

# Calculate similarity and predict plagiarism
predictions = []

#print(df.head(2))
ignored_idx = []

for index, row in df.iterrows():
    similarity = calculate_similarity(row['text1'], row['text2'])
    if similarity is None:
      df.at[index,'is_plagiarized'] = np.nan
      continue
    is_plagiarized = int(similarity > threshold)
    predictions.append(is_plagiarized)

# Evaluate model
predicted = np.array(predictions)
actual = df['is_plagiarized'].dropna()
actual=actual.values

accuracy = accuracy_score(actual, predicted)
f1 = f1_score(actual, predicted)

Text1="The paint on the canvas looks vibrant."
Text2="They paint the wall with vibrant colors."
preprocessed_text1 = preprocess_text(Text1)
preprocessed_text2 = preprocess_text(Text2)
print(preprocessed_text1)
print(preprocessed_text2)
similarity = calculate_similarity(preprocessed_text1, preprocessed_text2)
if similarity is None:
  similarity = default_similarity_value
is_plagiarized = int(similarity > threshold)
similarity_percentage = similarity * 100
output = 'Plagiarized' if is_plagiarized == 1 else 'Not Plagiarized'
print(f"Similarity for the two input documents is {output} with a similarity score of: {similarity_percentage:.2f}%")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


paint canvas look vibrant
paint wall vibrant color
Similarity for the two input documents is Not Plagiarized with a similarity score of: 33.61%
Accuracy: 1.0
F1 Score: 1.0


## **MACHINE LEARNING MODEL**

***SVM and Random Forest Classifier with custom dataset***

In [13]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
import numpy as np
import nltk

# Ensure you have the necessary packages
nltk.download('punkt')

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/plagiarism_dataset.csv')  # Make sure to provide the correct path to your CSV file

# Preprocessing: Tokenize the text
df['text1_tokens'] = df['text1'].apply(lambda x: word_tokenize(x.lower()))
df['text2_tokens'] = df['text2'].apply(lambda x: word_tokenize(x.lower()))

# Combine all tokens for Word2Vec training
all_tokens = pd.concat([df['text1_tokens'], df['text2_tokens']]).tolist()

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=all_tokens, vector_size=100, window=5, min_count=1, workers=4)

# Define function to create a feature vector for each text by averaging all word vectors
def document_vector(word_list, model):
    # remove out-of-vocabulary words
    word_list = [word for word in word_list if word in model.wv.index_to_key]
    if len(word_list) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(model.wv[word_list], axis=0)

# Create vectors for each text
df['text1_vector'] = df['text1_tokens'].apply(lambda x: document_vector(x, word2vec_model))
df['text2_vector'] = df['text2_tokens'].apply(lambda x: document_vector(x, word2vec_model))

# Prepare feature vectors for model training (subtracting vectors to get the difference)
X = np.array(df.apply(lambda row: row['text1_vector'] - row['text2_vector'], axis=1).tolist())
y = df['is_plagiarized'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM classifier
svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)

# Train Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

# Make predictions
svm_predictions = svm_clf.predict(X_test)
rf_predictions = rf_clf.predict(X_test)

# Evaluation
print("SVM Classifier Report")
print(classification_report(y_test, svm_predictions))

print("Random Forest Classifier Report")
print(classification_report(y_test, rf_predictions))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


SVM Classifier Report
              precision    recall  f1-score   support

           0       1.00      0.31      0.48       191
           1       0.70      1.00      0.83       309

    accuracy                           0.74       500
   macro avg       0.85      0.66      0.65       500
weighted avg       0.82      0.74      0.69       500

Random Forest Classifier Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       191
           1       1.00      1.00      1.00       309

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



In [14]:
# Function to preprocess and tokenize new text
def preprocess_and_tokenize(text):
    return word_tokenize(text.lower())

# Function to create a feature vector for prediction
def create_feature_vector(text1, text2, model):
    text1_vector = document_vector(preprocess_and_tokenize(text1), model)
    text2_vector = document_vector(preprocess_and_tokenize(text2), model)
    return text1_vector - text2_vector

# Two new input texts
new_text1 = "An example of text to be checked for plagiarism."
new_text2 = "A text that is to be compared with the previous one for plagiarism."

# Create feature vector for the new texts
new_feature_vector = create_feature_vector(new_text1, new_text2, word2vec_model).reshape(1, -1)

# Predict with SVM
svm_prediction = svm_clf.predict(new_feature_vector)
print(f"SVM Prediction for plagiarism: {'Plagiarized' if svm_prediction[0] == 1 else 'Not Plagiarized'}")

# Predict with Random Forest
rf_prediction = rf_clf.predict(new_feature_vector)
print(f"Random Forest Prediction for plagiarism: {'Plagiarized' if rf_prediction[0] == 1 else 'Not Plagiarized'}")


SVM Prediction for plagiarism: Plagiarized
Random Forest Prediction for plagiarism: Not Plagiarized


**SVM and Random Forest Classifier with custom dataset(still in progress)**

In [ ]:
from google.colab import drive
import os
import xml.etree.ElementTree as ET
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

nltk.download('punkt')  # Make sure this is uncommented if 'punkt' tokenizer is not already downloaded

def read_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def parse_xml_for_plagiarism(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    plagiarism_cases = []
    for feature in root.iter('feature'):
        if feature.attrib.get('type') == 'plagiarism':
            plagiarism_cases.append({
                'source_offset': int(feature.attrib['source_offset']),
                'source_length': int(feature.attrib['source_length']),
                'suspicious_offset': int(feature.attrib['this_offset']),
                'suspicious_length': int(feature.attrib['this_length']),
                'source_reference': feature.attrib['source_reference']
            })
    return plagiarism_cases

# Define the base path to the dataset
base_path = 'https://drive.google.com/drive/folders/1gwU9kIeii-VLU7SO8pi-ZKG12kXZKhk2?usp=sharing'

# Define paths for suspicious and source documents
suspicious_path = os.path.join(base_path, 'suspicious-document')
source_path = os.path.join(base_path, 'source-document')

# Placeholder for data
text_pairs = []

# Iterate through the dataset directory
for root, dirs, files in os.walk(suspicious_path):
    for file in files:
        if file.endswith('.txt'):
            suspicious_file_path = os.path.join(root, file)
            suspicious_text = read_text(suspicious_file_path)

            # Corresponding XML file
            xml_file_path = suspicious_file_path.replace('.txt', '.xml')
            plagiarism_cases = parse_xml_for_plagiarism(xml_file_path)

            # For each case of plagiarism
            for case in plagiarism_cases:
                # Calculate the character positions for the plagiarized section
                start_pos = case['suspicious_offset']
                end_pos = start_pos + case['suspicious_length']
                suspicious_excerpt = suspicious_text[start_pos:end_pos]

                # Read the corresponding source text
                source_file_path = os.path.join(source_path, case['source_reference'] + '.txt')
                source_text = read_text(source_file_path)
                source_start_pos = case['source_offset']
                source_end_pos = source_start_pos + case['source_length']
                source_excerpt = source_text[source_start_pos:source_end_pos]

                # Tokenize and preprocess both excerpts
                suspicious_tokens = word_tokenize(suspicious_excerpt)
                source_tokens = word_tokenize(source_excerpt)

                # Append to the dataset (token lists can be directly used for Word2Vec training)
                text_pairs.append((source_tokens, suspicious_tokens))

# Now text_pairs contains tokenized source and suspicious excerpts ready for vectorization

# The next steps would involve vectorizing the text pairs and training your machine learning models,
# similar to the previously provided examples.

# Continue with Word2Vec training, vectorization, and machine learning model training...

# Flatten the list of tokenized texts to train Word2Vec
flat_tokenized_texts = [token for pair in text_pairs for text in pair for token in text]

# Train a Word2Vec model on the corpus
word2vec_model = Word2Vec(sentences=flat_tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Define a function to create a feature vector for a text pair by averaging all word vectors
def feature_vector(text_pair, model):
    source_vector = np.mean([model.wv[token] for token in text_pair[0] if token in model.wv], axis=0)
    suspicious_vector = np.mean([model.wv[token] for token in text_pair[1] if token in model.wv], axis=0)
    return np.abs(source_vector - suspicious_vector)

# Create feature vectors for the dataset
feature_vectors = np.array([feature_vector(pair, word2vec_model) for pair in text_pairs])

# Define labels for the dataset: 1 for plagiarism, 0 for non-plagiarism
labels = [1 if pair else 0 for pair in text_pairs]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Train a SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

# Make predictions with both classifiers
rf_predictions = rf_classifier.predict(X_test)
svm_predictions = svm_classifier.predict(X_test)

# Evaluate the classifiers
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_predictions))

print("SVM Classification Report:")
print(classification_report(y_test, svm_predictions))



MessageError: Error: credential propagation was unsuccessful